## Environment Setting
Google drive mount (for Colab users) and package importing.
You can optionally work on a transformer part.

In [ ]:
# For Colab users
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import sys
sys.path.insert(0,'/content/drive/{path to project directory}')

In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm
import random
import json

from data_utils import MLDataset, collate_fn
from modeling import Seq2SeqModel # You can import your custom model classes from modeling.py

## (Optional) Sample Visualization
You can see actual sample images and correct answers. Additional matplotlib package is needed.

In [ ]:
# Just for reference: see actual samples
import matplotlib.pyplot as plt

id_to_char = {}
alphabets = "abcdefghijklmnopqrstuvwxyz"
for i, c in enumerate(alphabets):
    id_to_char[i+1] = c

In [ ]:
# Just for reference: see actual samples
idx = 1234
sample = np.load(f'./data_final/imgs/train/{idx}.npy')
with open('./data_final/labels/train.json', 'r') as f:
    sample_target = json.load(f)[str(idx)]
    
tgt_char = ""
for i in sample_target:
    tgt_char += id_to_char[i]


print(f"Answer: {tgt_char} ({sample_target})")
print("Input image sequence:")

plt.figure(figsize=(5, len(sample)))
for i, img in enumerate(sample):    
    plt.subplot(1, len(sample), i+1)
    plt.axis("off")
    plt.imshow(img)

## Device and seed setting

In [ ]:
assert torch.cuda.is_available()

# Use 0th GPU for training
torch.cuda.set_device(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# fix random seed to increase reproducibility
# NOTE: Do not modify here!
NUM_CLASSES = 26 + 2 # 26 alphabets + 1 padding index + 1 <s> token (start token)

random_seed = 7
torch.manual_seed(random_seed)
os.environ['PYTHONHASHSEED'] = str(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)
torch.cuda.manual_seed(random_seed)

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
# %env CUBLAS_WORKSPACE_CONFIG=:16:8

def seed_worker(worker_seed):
    np.random.seed(worker_seed)
    random.seed(worker_seed)

num_workers = 8

## Model loading and training

In [ ]:
# NOTE: modify path and batch size for your setting
# NOTE: you can apply custom preprocessing to the training data

BATCH_SIZE = 128

train_ds = MLDataset('data_final/imgs/train', 'data_final/labels/train.json')
valid_ds = MLDataset('data_final/imgs/valid_normal', 'data_final/labels/valid_normal.json')
challenge_ds = MLDataset('data_final/imgs/valid_challenge', 'data_final/labels/valid_challenge.json')

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=False)
challenge_dl = DataLoader(challenge_ds, batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=False)

In [ ]:
# You can add or modify your Seq2SeqModel's hyperparameter (keys and values)
kwargs = {
    'hidden_dim': 32,
    'n_rnn_layers': 2,
    'rnn_dropout': 0.5
}

In [ ]:
model = Seq2SeqModel(num_classes=NUM_CLASSES, **kwargs).to(device)
print(model)
##############################################################################
#                          IMPLEMENT YOUR CODE                               #
##############################################################################
model_optim =
loss_fn = 
# NOTE: you can define additional components like lr_scheduler, ...
##############################################################################
#                          END OF YOUR CODE                                  #
##############################################################################

In [ ]:
# NOTE: you can freely modify or add training hyperparameters
print_interval = 1000
max_epoch = 20

In [ ]:
def train(model, model_optim, loss_fn, max_epoch, train_dl, valid_dl, load_path=None, save_path='./model.pt'):
    ##############################################################################
    #                          IMPLEMENT YOUR CODE                               #
    ##############################################################################
    # Implement your train function
    
    ##############################################################################
    #                          END OF YOUR CODE                                  #
    ##############################################################################

    return

In [ ]:
load_path = None
train(model, model_optim, loss_fn, max_epoch, train_dl, valid_dl, load_path=load_path, save_path='./model.pt')

## Model evaluation

In [ ]:
kwargs_generate = {
    # you can add arguments for your model's generate function
}

In [ ]:
# Do not modify this cell!

def eval(dataloader, model_path):
    state = torch.load(model_path)
    model.load_state_dict(state["model"])
    model.eval()

    id_to_char = {}
    id_to_char[0] = "<pad>"
    id_to_char[27] = "<s>"
    alphabets = "abcdefghijklmnopqrstuvwxyz"
    for i, c in enumerate(alphabets):
        id_to_char[i+1] = c

    results = []
    labels = []    
    for batch_idx, (data, target, lengths) in enumerate(tqdm(dataloader)):       
        data = data.to(device) # (B, T, H, W, C)
        target = target.to(device) # (B, T)
        
        # start tokens should be located at the first position of the decoder input
        start_tokens = (torch.ones([target.size(0), 1]) * 27).to(torch.long).to(device)
        with torch.no_grad():
            generated_tok = model.generate(data, lengths, start_tokens, **kwargs_generate) # (B, T)
            
        for i in range(generated_tok.size(0)):
            decoded = ""
            for j in generated_tok[i][:lengths[i].int()].tolist():
                decoded += id_to_char[j]
            results.append(decoded)
    
            decoded = ""
            for j in target[i][:lengths[i].int()].tolist():
                decoded += id_to_char[j]
            labels.append(decoded)
        
    corrects = []
    for i in range(len(results)):
        if results[i] == labels[i]:
            corrects.append(1)
        else:
            corrects.append(0)
    print("Accuracy: %.5f" % (sum(corrects) / len(corrects)))

    return results, labels

In [ ]:
# load and evaluate your model
load_path = './model.pt'
print("Evaluation with validation set")
results, labels = eval(valid_dl, load_path)

print("Evaluation with chllenge set")
results, labels = eval(challenge_dl, load_path)